In [2]:
# package imported
import torch
import numpy as np
import json
import re
import torch.nn as nn

# file path
file_path="E:\\AAA\\nlp\\AA_my_poem_generator\\songci\\data.json"
voc_path="E:\\AAA\\nlp\\AA_my_poem_generator\\voc\\"
model_path="E:\\AAA\\nlp\\AA_my_poem_generator\\checkpoints_songci\\sc"

#poem selecting
min_len=10
max_len=200


# embedding setting
word2id = {}  # 字典转换，汉字转化为相应的index
id2word = {}  # index转换回汉字

# training setting
lr = 0.001
epoch = 20  # 训练整套数据的次数
embedding_dim = 128  # 词向量的维度
hidden_dim = 256 # 隐层的维度
batch_size = 128  # 批处理的量


In [3]:
#load data

In [4]:
with open(voc_path+"word2id.json") as f0:
    word2id=json.load(f0)
with open(voc_path+"id2word.json") as f1:
    id2word=json.load(f1)   

In [5]:
print(word2id,id2word)

{'坼': 0, '卉': 1, '堵': 2, '尫': 3, '如': 4, 'ō': 5, '郡': 6, '匝': 7, '业': 8, '制': 9, '玲': 10, '厓': 11, '蓖': 12, '藿': 13, '砂': 14, '帘': 15, '旒': 16, '飓': 17, '玺': 18, '众': 19, '盈': 20, '钥': 21, '蛰': 22, '拐': 23, '枨': 24, '为': 25, '撒': 26, '鹊': 27, '鳞': 28, '男': 29, '垆': 30, '怪': 31, '溥': 32, '佗': 33, '叙': 34, '雁': 35, '荐': 36, '豺': 37, '独': 38, '沔': 39, '炮': 40, '彀': 41, '菸': 42, '清': 43, '俊': 44, '柴': 45, '奏': 46, '查': 47, '身': 48, '门': 49, '妫': 50, '围': 51, '仔': 52, '鼓': 53, '枳': 54, '１': 55, '翠': 56, '搂': 57, '美': 58, '菅': 59, '缤': 60, '坝': 61, '苟': 62, '阐': 63, '锝': 64, '灼': 65, '带': 66, '砻': 67, '箜': 68, '赴': 69, '推': 70, '倚': 71, '冶': 72, '伎': 73, '浅': 74, '台': 75, '窥': 76, '酌': 77, '毁': 78, '猫': 79, '缉': 80, '谷': 81, '莽': 82, '舍': 83, '裳': 84, '殆': 85, '怆': 86, '嫩': 87, '辁': 88, '凿': 89, '纭': 90, '竺': 91, '坟': 92, '潺': 93, '记': 94, '淑': 95, '挂': 96, '算': 97, '粤': 98, '蒙': 99, '鄱': 100, '让': 101, '啄': 102, '黟': 103, '闰': 104, '轻': 105, '媾': 106, '集': 107, '泾': 108, '蝤': 109, '蛜': 110,

In [6]:
data=torch.load('E:\\AAA\\nlp\\AA_my_poem_generator\\songci\\tensor_data.pt')

In [7]:
data

tensor([[5295, 5295, 5295,  ..., 4224,  436, 5293],
        [5295, 5295, 5295,  ..., 2763,  436, 5293],
        [5295, 5295, 5295,  ..., 1251,  436, 5293],
        ...,
        [5295, 5295, 5295,  ...,  594,  436, 5293],
        [5295, 5295, 5295,  ..., 4062,  436, 5293],
        [5295, 5295, 5295,  ...,  379,  436, 5293]], dtype=torch.int32)

In [8]:
#输出中间结果的测试代码
def generate(model, start_words, ix2word, word2ix, prefix_words=None):
    
    results = list(start_words) #首先将结果置为list of start words
    start_word_len = len(start_words)
    input = t.Tensor([word2ix['S']]).view(1, 1).long() #第一个input为开始符号'S'
    #if opt.use_gpu: input = input.cuda()
    hidden = None #传入的hidden是None，在模型内部初始化为0

    if prefix_words:
        for word in prefix_words:
            output, hidden = model(input, hidden)
            input = input.data.new([word2id[word]]).view(1, 1)

    for i in range(200):
        output, hidden = model(input, hidden)

        if i < start_word_len:
            w = results[i]
            input = input.data.new([word2id[w]]).view(1, 1)
        else:
            top_index = output.data[0].topk(1)[1][0].item()
            w = id2word[str(top_index)]
            results.append(w)
            input = input.data.new([top_index]).view(1, 1)
        if w == 'E':
            del results[-1]
            break
    return results

In [9]:
import torch.nn as nn

class POEM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(POEM, self).__init__()
        self.hidden_dim = hidden_dim

        self.encoder = nn.Embedding(vocab_size, embedding_dim) # one hot representation to embedding dim

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,num_layers=2) #构造有两个隐层的LSTM

        # The linear layer that maps from hidden state space to tag space
        self.decoder = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input, hidden=None):
        # 向前遍历，产生一系列的output，最后将output整合与target进行对比

        seq_len, batch_size= input.size() #input是batch的形式
        if hidden is None:
            h = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
            c = input.data.new(2, batch_size, self.hidden_dim).fill_(0).float()
        else:
            h,c=hidden
        encoded=self.encoder(input)
        output, hidden =self.lstm(encoded, (h,c))

        output=self.decoder(output.view(seq_len*batch_size, -1))
        return output,hidden



In [10]:
import torch as t
from torchnet import meter

def train(Epoch,data,word2id,id2word,index):


    # 获取数据

    dataloader = t.utils.data.DataLoader(data,
                                         batch_size=batch_size,
                                         shuffle=True)

    # 模型定义

    model = POEM(len(word2id), embedding_dim, hidden_dim)
    if index!=0:
        model.load_state_dict(torch.load(model_path+'_'+str(index-1)+'.pth'))
    optimizer = t.optim.Adam(model.parameters(), lr=0.0001)#lr:学习率
    criterion = nn.CrossEntropyLoss()
    loss_meter = meter.AverageValueMeter()
    for epoch in range(Epoch):
        loss_meter.reset()
        for ii, data_ in enumerate(dataloader):

            # 训练
            data_ = data_.long().transpose(1, 0).contiguous()
            optimizer.zero_grad()
            input_, target = data_[:-1, :], data_[1:, :]#target:y_train, 将整个序列右移

            output, _ = model(input_)
            print('input:',input_)
            print('target:',target)
            print('output:',output)
            loss = criterion(output, target.view(-1))
            print('loss:',loss)
            loss.backward()
            optimizer.step()

            loss_meter.add(loss.item())

            # 打印诗歌
            if (1 + ii) % 20 == 0:
                # 诗歌原文
                gen_poetries = []
                # 分别以这几个字作为诗歌的第一个字，生成8首诗
                for word in list(u'春江花月夜凉如水'):
                    gen_poetry = ''.join(generate(model, word, id2word, word2id))
                    gen_poetries.append(gen_poetry)
                print(gen_poetries)

        t.save(model.state_dict(), '%s_%s.pth' % (model_path, epoch+index))

In [33]:
train(2,data,word2id,id2word,0 )


0it [00:00, ?it/s]
1it [00:10, 10.69s/it]
2it [00:18,  9.88s/it]
3it [00:26,  9.37s/it]
4it [00:34,  9.00s/it]
5it [00:43,  8.85s/it]
6it [00:52,  8.77s/it]
7it [01:00,  8.70s/it]
8it [01:09,  8.66s/it]
9it [01:17,  8.65s/it]
10it [01:26,  8.63s/it]
11it [01:34,  8.52s/it]
12it [01:42,  8.42s/it]
13it [01:50,  8.34s/it]
14it [01:59,  8.36s/it]
15it [02:07,  8.40s/it]
16it [02:16,  8.40s/it]
17it [02:24,  8.37s/it]
18it [02:33,  8.51s/it]
19it [02:42,  8.68s/it]

['春                                                                                                                                                                                                       ', '江                                                                                                                                                                                                       ', '花                                                                                                                                                                                                       ', '月                                                                                                                                                                                                       ', '夜                                                                                                                                                                                     


20it [02:51,  8.89s/it]
21it [03:00,  8.76s/it]
22it [03:09,  8.78s/it]
23it [03:17,  8.70s/it]
24it [03:26,  8.61s/it]
25it [03:34,  8.62s/it]
26it [03:43,  8.59s/it]
27it [03:52,  8.70s/it]
28it [04:00,  8.65s/it]
29it [04:09,  8.66s/it]
30it [04:17,  8.59s/it]
31it [04:26,  8.57s/it]
32it [04:34,  8.52s/it]
33it [04:43,  8.53s/it]
34it [04:51,  8.45s/it]
35it [05:00,  8.62s/it]
36it [05:09,  8.66s/it]
37it [05:18,  8.69s/it]
38it [05:27,  8.78s/it]
39it [05:35,  8.80s/it]

['春                                                                                                                                                                                                       ', '江。娈演嶢磕磕。万屹璞演磕。娴泸泸渥磕磕磕骂。娴渥跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣', '花                                                                                                                                                                                                       ', '月。万演蓄蓄蓄淞跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣跣', '夜。演娴磕渥磕万                                                                                                                                                                              


40it [05:46,  9.19s/it]
41it [05:55,  9.14s/it]
42it [06:03,  9.05s/it]
43it [06:12,  8.99s/it]
44it [06:21,  8.95s/it]
45it [06:30,  8.87s/it]
46it [06:39,  8.85s/it]
47it [06:48,  8.88s/it]
48it [06:57,  8.90s/it]
49it [07:06,  8.98s/it]
50it [07:15,  8.95s/it]
51it [07:23,  8.94s/it]
52it [07:32,  8.88s/it]
53it [07:41,  8.89s/it]
54it [07:49,  8.72s/it]
55it [07:58,  8.68s/it]
56it [08:07,  8.66s/it]
57it [08:16,  8.76s/it]
58it [08:25,  8.88s/it]
59it [08:33,  8.81s/it]

...


100it [15:06,  8.92s/it]
101it [15:15,  8.75s/it]
102it [15:23,  8.65s/it]
103it [15:32,  8.61s/it]
104it [15:40,  8.65s/it]
105it [15:49,  8.61s/it]
106it [15:58,  8.66s/it]
107it [16:07,  8.73s/it]
108it [16:15,  8.66s/it]
109it [16:24,  8.66s/it]
110it [16:32,  8.66s/it]
111it [16:41,  8.71s/it]
112it [16:50,  8.70s/it]
113it [16:59,  8.73s/it]
114it [17:08,  8.77s/it]
115it [17:17,  9.01s/it]
116it [17:26,  9.05s/it]
117it [17:35,  8.92s/it]
118it [17:44,  8.88s/it]
119it [17:52,  8.81s/it]

['春晏。一鳏涝涝涝涝滤泸惫                                                                                                                                                                                            ', '江涝泸泸泸扼                                                                                                                                                                                                  ', '花演。不教鳏涝涝涝滤脏绔亚                                                                                                                                                                                           ', '月魉倜                                                                                                                                                                                                     ', '夜涝脏涝涝涝涝涝涝泸泸泸磕                                                                                                                                                                         


120it [18:02,  9.11s/it]
121it [18:11,  8.94s/it]
122it [18:19,  8.81s/it]
123it [18:28,  8.76s/it]
124it [18:37,  8.78s/it]
125it [18:46,  8.98s/it]
126it [18:55,  8.93s/it]
127it [19:04,  8.95s/it]
128it [19:14,  9.13s/it]
129it [19:22,  8.93s/it]
130it [19:31,  8.98s/it]
131it [19:40,  8.89s/it]
132it [19:49,  8.93s/it]
133it [19:58,  8.89s/it]
134it [20:07,  8.98s/it]
135it [20:17,  9.32s/it]
136it [20:26,  9.32s/it]
137it [20:35,  9.29s/it]
138it [20:45,  9.40s/it]
139it [20:54,  9.35s/it]

['春。一鳏鳏涝涝涝涝泸{                                                                                                                                                                                             ', '江涝泸泸脏泸涝涝涝涝涝涝涝涝涝涝泸泸泸磕                                                                                                                                                                                    ', '花演                                                                                                                                                                                                      ', '月倜。恐鳏涝涝涝涝涝泸{                                                                                                                                                                                            ', '夜涝涝泸泸泸泸扼                                                                                                                                                                              


140it [21:05,  9.68s/it]
141it [21:14,  9.60s/it]
142it [21:23,  9.47s/it]
143it [21:33,  9.42s/it]
144it [21:42,  9.25s/it]
145it [21:50,  9.16s/it]
146it [22:00,  9.13s/it]
147it [22:09,  9.24s/it]
148it [22:19,  9.35s/it]
149it [22:28,  9.33s/it]
150it [22:37,  9.38s/it]
151it [22:46,  9.19s/it]
152it [22:55,  9.16s/it]
153it [23:05,  9.31s/it]
154it [23:15,  9.53s/it]
155it [23:24,  9.45s/it]
156it [23:33,  9.37s/it]
157it [23:39,  8.31s/it]


In [10]:
train(5,data,word2id,id2word,2)

19it [02:43,  8.58s/it]

['春。一一一人、一人人。', '江教不不不人人。', '花事。', '月倜                                                                                                                                                                                                      ', '夜演。', '凉。一一一人人、人人。', '如谁                                                                                                                                                                                                      ', '水                                                                                                                                                                                                       ']


39it [05:38,  8.69s/it]

['春。一一一人、一人、一人、一人。', '江教不不不人、一人、一人、一人。', '花。一一一不人、一人、一人。', '月晏。一一一不人、一人、一人。', '夜教。一一一不人、一人、一人。', '凉。', '如谁                                                                                                                                                                                                      ', '水。']


59it [08:36,  8.84s/it]

...

119it [18:18,  9.35s/it]

['春暮                                                                                                                                                                                                      ', '江北水                                                                                                                                                                                                     ', '花悴。', '月下，一番花开酒。', '夜蓉雨。', '凉岛。', '如缈                                                                                                                                                                                                      ', '水晶台月，一枝风月，一夜风流。']


139it [21:22,  9.09s/it]

['春暮                                                                                                                                                                                                      ', '江北水                                                                                                                                                                                                     ', '花悴。', '月下，一番花、一点春风。', '夜蓉雨。', '凉岛。', '如缈                                                                                                                                                                                                      ', '水晶台月，一枝花下，一片春风。']


157it [23:58,  8.12s/it]


In [ ]:
loss_vec=train(5,data,word2id,id2word,7)

input: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [3578, 4866, 3331,  ..., 5138, 5183,  422],
        [4096, 3563,  754,  ..., 1505, 5183, 3547],
        [ 436,  436,  436,  ...,  436,  436,  436]])
target: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [4096, 3563,  754,  ..., 1505, 5183, 3547],
        [ 436,  436,  436,  ...,  436,  436,  436],
        [5293, 5293, 5293,  ..., 5293, 5293, 5293]])
output: tensor([[-2.4633, -4.0425, -3.8949,  ..., -5.4169,  6.0162, 12.9514],
        [-2.4633, -4.0425, -3.8949,  ..., -5.4169,  6.0162, 12.9514],
        [-2.4633, -4.0425, -3.8949,  ..., -5.4169,  6.0162, 12.9514],
        ...,
        [-4.2152, -3.8265, -4.3285,  ..., 10.3279, -2.2930, -2.5068],
        [-5.1368, -5.3473, -5.46

loss: tensor(1.8206, grad_fn=<NllLossBackward>)
input: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [2210, 4825,  366,  ..., 3990, 5295, 5151],
        [1452, 1240,  249,  ..., 3667, 5295, 3695],
        [ 436,  436,  436,  ...,  436, 5294,  436]])
target: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [1452, 1240,  249,  ..., 3667, 5295, 3695],
        [ 436,  436,  436,  ...,  436, 5294,  436],
        [5293, 5293, 5293,  ..., 5293, 5293, 5293]])
output: tensor([[-2.5002, -4.0392, -3.9297,  ..., -5.4187,  5.9653, 13.0100],
        [-2.5002, -4.0392, -3.9297,  ..., -5.4187,  5.9653, 13.0100],
        [-2.5002, -4.0392, -3.9297,  ..., -5.4187,  5.9653, 13.0100],
        ...,
        [-4.8701, -4.2019, -4.6208,  ..., 11.0199, -2.

loss: tensor(1.8668, grad_fn=<NllLossBackward>)
input: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [3010, 4716, 1947,  ...,  948, 4987, 1271],
        [3557, 1474,  321,  ..., 3480, 3279, 3077],
        [ 436,  436,  436,  ...,  436,  436,  436]])
target: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [3557, 1474,  321,  ..., 3480, 3279, 3077],
        [ 436,  436,  436,  ...,  436,  436,  436],
        [5293, 5293, 5293,  ..., 5293, 5293, 5293]])
output: tensor([[-2.4929, -4.0228, -3.9504,  ..., -5.4202,  5.9696, 13.0120],
        [-2.4929, -4.0228, -3.9504,  ..., -5.4202,  5.9696, 13.0120],
        [-2.4929, -4.0228, -3.9504,  ..., -5.4202,  5.9696, 13.0120],
        ...,
        [-4.2668, -3.8407, -4.7211,  ..., 10.2608, -2.

loss: tensor(1.8914, grad_fn=<NllLossBackward>)
input: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [2792,  740, 1395,  ..., 2745, 4087, 4204],
        [3997, 4737, 5253,  ..., 2964,  411, 3087],
        [ 436,  436,  436,  ...,  436,  436,  436]])
target: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [3997, 4737, 5253,  ..., 2964,  411, 3087],
        [ 436,  436,  436,  ...,  436,  436,  436],
        [5293, 5293, 5293,  ..., 5293, 5293, 5293]])
output: tensor([[-2.4935, -4.0239, -3.9613,  ..., -5.4296,  5.9760, 13.0085],
        [-2.4935, -4.0239, -3.9613,  ..., -5.4296,  5.9760, 13.0085],
        [-2.4935, -4.0239, -3.9613,  ..., -5.4296,  5.9760, 13.0085],
        ...,
        [-4.2610, -4.0013, -5.5912,  ..., 10.6342, -2.

loss: tensor(1.8940, grad_fn=<NllLossBackward>)
input: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [2219,   45, 2064,  ...,    4, 2947, 2414],
        [1306, 5033, 4263,  ..., 3852, 3332, 4679],
        [ 436,  436,  436,  ...,  436,  436,  436]])
target: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [1306, 5033, 4263,  ..., 3852, 3332, 4679],
        [ 436,  436,  436,  ...,  436,  436,  436],
        [5293, 5293, 5293,  ..., 5293, 5293, 5293]])
output: tensor([[-2.4966, -4.0316, -3.9622,  ..., -5.4280,  5.9732, 13.0151],
        [-2.4966, -4.0316, -3.9622,  ..., -5.4280,  5.9732, 13.0151],
        [-2.4966, -4.0316, -3.9622,  ..., -5.4280,  5.9732, 13.0151],
        ...,
        [-4.3256, -3.8702, -5.3917,  ...,  9.1734, -2.

...

loss: tensor(1.8196, grad_fn=<NllLossBackward>)
input: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [3331, 2057,  709,  ..., 4526, 1131, 1399],
        [2809, 3547, 4166,  ..., 2995, 1877, 2809],
        [ 436,  436,  436,  ...,  436,  436,  436]])
target: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [2809, 3547, 4166,  ..., 2995, 1877, 2809],
        [ 436,  436,  436,  ...,  436,  436,  436],
        [5293, 5293, 5293,  ..., 5293, 5293, 5293]])
output: tensor([[-2.6893, -3.7475, -3.8122,  ..., -5.7990,  5.8280, 13.2932],
        [-2.6893, -3.7475, -3.8122,  ..., -5.7990,  5.8280, 13.2932],
        [-2.6893, -3.7475, -3.8122,  ..., -5.7990,  5.8280, 13.2932],
        ...,
        [-4.5347, -3.8158, -5.1394,  ..., 11.2204, -3.

loss: tensor(1.8431, grad_fn=<NllLossBackward>)
input: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [5051,  740,  321,  ...,  825, 2713, 1968],
        [3951, 5219,  961,  ...,  831, 2964, 2602],
        [ 436,  436,  436,  ...,  436,  436,  436]])
target: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [3951, 5219,  961,  ...,  831, 2964, 2602],
        [ 436,  436,  436,  ...,  436,  436,  436],
        [5293, 5293, 5293,  ..., 5293, 5293, 5293]])
output: tensor([[-2.6868, -3.7443, -3.8024,  ..., -5.8160,  5.8306, 13.2961],
        [-2.6868, -3.7443, -3.8024,  ..., -5.8160,  5.8306, 13.2961],
        [-2.6868, -3.7443, -3.8024,  ..., -5.8160,  5.8306, 13.2961],
        ...,
        [-4.4812, -3.9763, -5.5978,  ..., 12.0884, -3.

loss: tensor(1.8371, grad_fn=<NllLossBackward>)
input: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [ 965, 3878, 4265,  ..., 4905, 2947, 3321],
        [1086,  173, 4526,  ...,  825,  320, 2058],
        [ 436,  436,  436,  ...,  436,  436,  436]])
target: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [1086,  173, 4526,  ...,  825,  320, 2058],
        [ 436,  436,  436,  ...,  436,  436,  436],
        [5293, 5293, 5293,  ..., 5293, 5293, 5293]])
output: tensor([[-2.6818, -3.7413, -3.7820,  ..., -5.8328,  5.8271, 13.3045],
        [-2.6818, -3.7413, -3.7820,  ..., -5.8328,  5.8271, 13.3045],
        [-2.6818, -3.7413, -3.7820,  ..., -5.8328,  5.8271, 13.3045],
        ...,
        [-4.3005, -4.0927, -5.9944,  ..., 10.6763, -2.

loss: tensor(1.7498, grad_fn=<NllLossBackward>)
input: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [1989, 5169,    4,  ..., 3642, 3898, 4526],
        [ 948,  851, 1584,  ..., 4573,  321, 3068],
        [ 436,  436,  436,  ...,  436,  436,  436]])
target: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [ 948,  851, 1584,  ..., 4573,  321, 3068],
        [ 436,  436,  436,  ...,  436,  436,  436],
        [5293, 5293, 5293,  ..., 5293, 5293, 5293]])
output: tensor([[-2.6790, -3.7441, -3.7756,  ..., -5.8408,  5.8243, 13.3068],
        [-2.6790, -3.7441, -3.7756,  ..., -5.8408,  5.8243, 13.3068],
        [-2.6790, -3.7441, -3.7756,  ..., -5.8408,  5.8243, 13.3068],
        ...,
        [-4.7238, -3.9951, -5.6858,  ..., 12.2411, -2.

loss: tensor(1.9198, grad_fn=<NllLossBackward>)
loss: tensor(1.8344, grad_fn=<NllLossBackward>)
input: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [1081, 1940, 4987,  ...,    4,  321, 5295],
        [4888, 2629, 4559,  ...,  740, 2164, 5295],
        [ 436,  436,  436,  ...,  436,  436, 5294]])
target: tensor([[5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        [5295, 5295, 5295,  ..., 5295, 5295, 5295],
        ...,
        [4888, 2629, 4559,  ...,  740, 2164, 5295],
        [ 436,  436,  436,  ...,  436,  436, 5294],
        [5293, 5293, 5293,  ..., 5293, 5293, 5293]])
output: tensor([[-2.6761, -3.7527, -3.7751,  ..., -5.8425,  5.8352, 13.2987],
        [-2.6761, -3.7527, -3.7751,  ..., -5.8425,  5.8352, 13.2987],
        [-2.6761, -3.7527, -3.7751,  ..., -5.8425,  5.8352, 13.2987],
        ...,
      

In [ ]:
loss_vec=train(5,data,word2id,id2word,7)